# Cellpose 2.x Installation - WORKING VERSION for MyBinder

This notebook installs **Cellpose 2.x** with the classic API that works end-to-end.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/your-username/your-repo/HEAD?labpath=cellpose_working.ipynb)

## Features:
✅ Installs Cellpose 2.x (stable API)  
✅ Creates synthetic test data  
✅ Runs segmentation successfully  
✅ Shows beautiful results  
✅ Provides working code examples

## Step 1: Environment Setup

In [ ]:
import sys
import os
import subprocess
import time

print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"Platform: {sys.platform}")

def run_cmd(cmd, timeout=300):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return result.returncode == 0, result.stdout.strip(), result.stderr.strip()
    except subprocess.TimeoutExpired:
        return False, "", f"Timeout after {timeout}s"
    except Exception as e:
        return False, "", str(e)

print("\n✅ Environment setup complete!")

## Step 2: Install Cellpose 2.x and Dependencies

In [ ]:
print("🔧 Installing Cellpose 2.x and dependencies...")
print("This may take 3-4 minutes...")

# Install packages in order
packages = [
    "numpy",
    "scipy", 
    "matplotlib",
    "scikit-image",
    "opencv-python-headless",
    "tifffile",
    "torch",
    "torchvision",
    "cellpose==2.2.4"
]

successful = []
failed = []

for i, pkg in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] Installing {pkg}...", end=" ")
    success, stdout, stderr = run_cmd(f"pip install --quiet {pkg}")
    
    if success:
        print("✅")
        successful.append(pkg)
    else:
        print("❌")
        failed.append(pkg)
        if stderr:
            lines = stderr.split('\n')
            error_line = next((line for line in lines if 'ERROR' in line), lines[-1])
            print(f"    Error: {error_line[:80]}...")

print(f"\n📊 Installation Summary:")
print(f"  ✅ Successful: {len(successful)}/{len(packages)}")
print(f"  ❌ Failed: {len(failed)}")

if len(failed) > 0:
    print(f"  Failed packages: {failed}")

installation_ok = len(successful) >= 7
print(f"\nStatus: {'✅ READY' if installation_ok else '⚠️ PARTIAL'}")

## Step 3: Test Cellpose Import

In [ ]:
print("🔍 Testing Cellpose 2.x...")

try:
    import cellpose
    from cellpose import models, io, utils
    import numpy as np
    import matplotlib.pyplot as plt
    import tifffile
    
    print("✅ All imports successful")
    print(f"✅ Cellpose version: {cellpose.__version__}")
    
    # Test model initialization
    model = models.Cellpose(gpu=False, model_type='cyto')
    print("✅ Cellpose model initialized")
    
    print("\n🎉 CELLPOSE 2.x IS WORKING!")
    cellpose_working = True
    
except Exception as e:
    print(f"❌ Error: {e}")
    cellpose_working = False

# Configure matplotlib
if cellpose_working:
    import matplotlib
    matplotlib.use('Agg')
    plt.ioff()
    print("✅ Matplotlib configured")

print(f"\nCellpose Status: {'✅ WORKING' if cellpose_working else '❌ FAILED'}")

## Step 4: Create Test Data

In [ ]:
if cellpose_working:
    print("🔬 Creating synthetic test data...")
    
    from scipy import ndimage
    
    def create_cells(size=(512, 512), n_cells=12):
        img = np.zeros(size, dtype=np.float32)
        np.random.seed(42)
        
        for i in range(n_cells):
            cy = np.random.randint(50, size[0]-50)
            cx = np.random.randint(50, size[1]-50)
            radius = np.random.randint(18, 35)
            
            y, x = np.ogrid[:size[0], :size[1]]
            dist = np.sqrt((x - cx)**2 + (y - cy)**2)
            noise = np.random.normal(0, 3, size)
            mask = dist <= (radius + 0.3 * noise)
            
            intensity = np.random.uniform(160, 240)
            img[mask] = intensity
        
        img += np.random.normal(20, 8, size)
        img = ndimage.gaussian_filter(img, sigma=1.5)
        img = np.clip(img, 0, 255).astype(np.uint8)
        
        return img
    
    test_img = create_cells()
    tifffile.imwrite('test_cells.tif', test_img)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(test_img, cmap='gray')
    plt.title('Synthetic Test Cells', fontsize=14)
    plt.axis('off')
    plt.colorbar(shrink=0.8)
    plt.tight_layout()
    plt.savefig('test_preview.png', dpi=120, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Test image created: {test_img.shape}")
    print(f"   Range: {test_img.min()}-{test_img.max()}")
    test_data_ok = True
    
else:
    print("⚠️ Skipping test data - Cellpose not working")
    test_data_ok = False

## Step 5: Run Segmentation

In [ ]:
if cellpose_working and test_data_ok:
    print("🎯 Running Cellpose segmentation...")
    
    try:
        img = tifffile.imread('test_cells.tif')
        print(f"Loaded image: {img.shape}")
        
        print("Running segmentation...")
        start_time = time.time()
        
        masks, flows, styles, diams = model.eval(
            img, 
            diameter=None,
            channels=[0,0],
            flow_threshold=0.4,
            cellprob_threshold=0.0
        )
        
        end_time = time.time()
        n_cells = len(np.unique(masks)) - 1
        
        print(f"\n🎉 Segmentation completed!")
        print(f"  Time: {end_time - start_time:.1f} seconds")
        print(f"  Cells detected: {n_cells}")
        print(f"  Diameter: {diams:.1f} pixels")
        
        segmentation_ok = True
        
    except Exception as e:
        print(f"❌ Segmentation failed: {e}")
        segmentation_ok = False
        
else:
    print("⚠️ Skipping segmentation - prerequisites not met")
    segmentation_ok = False

print(f"\nSegmentation: {'✅ SUCCESS' if segmentation_ok else '❌ FAILED'}")

## Step 6: Visualize Results

In [ ]:
if segmentation_ok and 'masks' in locals():
    print("📊 Creating visualization...")
    
    from skimage import measure
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    # Masks
    axes[1].imshow(masks, cmap='nipy_spectral')
    axes[1].set_title(f'Masks ({n_cells} cells)')
    axes[1].axis('off')
    
    # Overlay
    axes[2].imshow(img, cmap='gray')
    axes[2].contour(masks, levels=[0.5], colors='red', linewidths=1.5)
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig('results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Save files
    tifffile.imwrite('masks.tif', masks.astype(np.uint16))
    outlines = utils.masks_to_outlines(masks)
    tifffile.imwrite('outlines.tif', outlines.astype(np.uint8) * 255)
    
    # Analysis
    props = measure.regionprops(masks, img)
    areas = [p.area for p in props]
    intensities = [p.mean_intensity for p in props]
    
    print(f"\n📈 Cell Analysis:")
    print(f"  Total cells: {len(areas)}")
    print(f"  Average area: {np.mean(areas):.1f} ± {np.std(areas):.1f} px²")
    print(f"  Size range: {min(areas):.0f} - {max(areas):.0f} px²")
    print(f"  Average intensity: {np.mean(intensities):.1f}")
    
    print(f"\n💾 Files saved:")
    print(f"  📄 results.png (visualization)")
    print(f"  📄 masks.tif (segmentation masks)")
    print(f"  📄 outlines.tif (cell outlines)")
    
    visualization_ok = True
    
else:
    print("⚠️ Skipping visualization - segmentation failed")
    visualization_ok = False

## Step 7: Usage Examples

In [ ]:
print("📚 CELLPOSE 2.x USAGE GUIDE")
print("=" * 50)

print("""
🔬 Basic Workflow:

from cellpose import models
import tifffile

# Load image
img = tifffile.imread('your_image.tif')

# Initialize model
model = models.Cellpose(gpu=False, model_type='cyto')

# Run segmentation
masks, flows, styles, diams = model.eval(
    img, 
    diameter=None,        # Auto-estimate
    channels=[0,0],       # Grayscale
    flow_threshold=0.4,   # Sensitivity
    cellprob_threshold=0.0
)

# Save results
tifffile.imwrite('masks.tif', masks)
""")

print("""
🎛️ Key Parameters:

• diameter=None: Auto-estimate (recommended)
• diameter=30: Set expected cell size
• flow_threshold=0.4: Lower = more cells
• cellprob_threshold=0.0: Higher = fewer cells
• channels=[0,0]: Grayscale image
• model_type: 'cyto', 'nuclei', 'cyto2'
""")

print("""
🔧 Available Models:

• 'cyto': General cell segmentation
• 'nuclei': Nuclear segmentation
• 'cyto2': Improved cytoplasm model
""")

if segmentation_ok:
    print(f"""
✅ Your Results:

• Successfully segmented {n_cells} cells
• Used Cellpose 2.x (stable, proven)
• Results ready for analysis
• Compatible with ImageJ, Python, R
""")

print("""
🚀 Next Steps:

1. Upload your own images
2. Adjust parameters for your data
3. Try different models
4. Extract quantitative measurements
5. Integrate with your workflow
""")

## Step 8: Final Summary

In [ ]:
print("🎉 CELLPOSE 2.x SETUP - FINAL REPORT")
print("=" * 50)

overall_success = all([
    installation_ok,
    cellpose_working, 
    test_data_ok,
    segmentation_ok,
    visualization_ok
])

print("\n📁 Generated Files:")
files = ['test_cells.tif', 'test_preview.png', 'results.png', 'masks.tif', 'outlines.tif']
for f in files:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024
        print(f"  ✅ {f} ({size:.1f} KB)")
    else:
        print(f"  ❌ {f} (missing)")

print("\n🔍 Component Status:")
components = [
    ("Package Installation", installation_ok),
    ("Cellpose Import", cellpose_working),
    ("Test Data Creation", test_data_ok),
    ("Segmentation", segmentation_ok),
    ("Visualization", visualization_ok)
]

for name, status in components:
    print(f"  {'✅' if status else '❌'} {name}")

if overall_success:
    print("\n🎉 STATUS: COMPLETE SUCCESS!")
    print("\n✅ Achievements:")
    print("  🔬 Cellpose 2.x fully functional")
    print("  🖼️ Test data created and segmented")
    if 'n_cells' in locals():
        print(f"  🔍 Detected {n_cells} cells successfully")
    print("  📊 Generated visualizations")
    print("  💾 Saved results in multiple formats")
    print("  📖 Provided usage examples")
    
    print("\n🎯 Ready for Research:")
    print("  • Upload your microscopy images")
    print("  • Adjust parameters for your cells")
    print("  • Try different Cellpose models")
    print("  • Extract quantitative data")
    print("  • Integrate with analysis pipelines")
    
    if 'end_time' in locals() and 'start_time' in locals():
        print(f"\n📈 Performance:")
        print(f"  ⏱️ Segmentation: {end_time - start_time:.1f}s")
        print(f"  💻 Platform: MyBinder (CPU)")
        print(f"  🧠 Model: Cellpose 2.x cyto")
    
else:
    print("\n⚠️ STATUS: INCOMPLETE")
    print("\n🔧 Try:")
    print("  1. Restart kernel and run all cells")
    print("  2. Check MyBinder resources")
    print("  3. Try a fresh session")
    print("  4. Contact instructors")

print("\n" + "=" * 50)
if overall_success:
    print("🔬✨ CELLPOSE 2.x READY FOR YOUR RESEARCH! ✨🔬")
else:
    print("🔧 TROUBLESHOOTING NEEDED")
print("=" * 50)